In [136]:
import pandas as pd
import re

In [137]:
leagues_prefixs = ["SA1", "SA2L"]
clubs_list = []
champions_list = []

In [138]:
def attendances_tranform():
  df_f = pd.read_csv("./RAW/ArabicSoccer/Attendances/Arabic_Attendances_2007_2023.csv")

  df = df_f[df_f['#'] >= 1]
  df = df[["Stadium", "Capacity", "Spectators", "Average", "Matches", "sold out", "Club", "Year"]]

  pattern = "\B[ANH][Llab][-]?[a-zA-Z]+(?:-[a-zA-Z]+)?\s?[SFClub]{1,4}"
    
  for i in range(len(df)):
      if "Al-Ansar" in df.iloc[i, 0]:
        df.iloc[i, 6] = "Al-Ansar"
        df.iloc[i, 0] = "Prince Mohammed bin Abdul Aziz Stadium"
      elif "Ohod Club" in df.iloc[i, 0]:
        df.iloc[i, 6] = "Ohod Club"
        df.iloc[i, 0] = "Prince Mohammed bin Abdul Aziz Stadium" 
      elif "Damac FC" in df.iloc[i, 0]:
        df.iloc[i, 6] = "Damac FC"
        df.iloc[i, 0] = "Damac Club Stadium"
      else:
        df.iloc[i, 6] = re.findall(pattern=pattern, string=df.iloc[i, 0])[0]
        df.iloc[i, 0] = re.sub(pattern=pattern, string=df.iloc[i, 0], repl="")

  df = df.reset_index()
  df["index"] = df.index + 1
  df['Matches'] = pd.to_numeric(df['Matches'], downcast='integer', errors='coerce')
  df = df.rename(columns={"index": "id"})
  return df

In [139]:
def champions_list_transform(league):
  for i in range(len(league)):
    df = pd.read_csv(f"./RAW/ArabicSoccer/Champions/{league[i]}_Champions.csv")
    df = df[["Season","Club", "League"]]
    df.rename(columns={"Club": "Clubs"}, inplace=True)
    if i == 2:
      df = df[:15]
      champions_list.append(df)
    else:
      df = df[:16]
      champions_list.append(df)

In [140]:
def teams_transform():
  for y in range(len(leagues_prefixs)):
    df = pd.read_csv(f"./RAW/ArabicSoccer/Teams/{leagues_prefixs[y]}_teams_2023.csv")
    df = df[["Club.1", "name","Squad", "ø age", "Foreigners", "ø market value", "League"]]
    df.rename(columns={"Club.1": "Clubs", "name": "Players", "Squad":"avg_age", "ø age":"Foreigners", "Foreigners": "market_value(€)", "ø market value": "total_market_value(€)"}, inplace=True)

    for i, row in df.iterrows():
      df.iloc[i, 4] = re.sub(pattern='[€mkK]', string=row["market_value(€)"], repl="")
      df.iloc[i, 5] = re.sub(pattern='[€mkK]', string=row["total_market_value(€)"], repl="")
      
    df = df[:-1]
    df["id"] = df.index + 1
    df['"Foreigners"'] = pd.to_numeric(df["Foreigners"], downcast='integer', errors='coerce')
    df = df[["id", "Clubs","Players", "avg_age", "Foreigners", "market_value(€)", "total_market_value(€)", "League"]]
    clubs_list.append(df)

    if y == 2:
      return 


In [141]:
df_attendances = attendances_tranform()
teams_transform()
df_teams = pd.concat(clubs_list) 
champions_list_transform(leagues_prefixs)
df_champions = pd.concat(champions_list)

In [142]:
df_teams

,id,Clubs,Players,avg_age,Foreigners,market_value(€),total_market_value(€),League
0,1,Al-Hilal SFC,29.0,27.9,9,8.93,259.00,SA1
1,2,Al-Ahli SFC,30.0,25.7,10,6.73,201.78,SA1
2,3,Al-Nassr FC,27.0,28.1,9,7.24,195.53,SA1
3,4,Al-Ittihad Club,32.0,27.2,10,4.17,133.30,SA1
4,5,Al-Shabab FC,25.0,27.4,9,2.78,69.38,SA1
5,6,Al-Ettifaq FC,25.0,26.4,9,2.67,66.83,SA1
6,7,Al-Taawoun FC,29.0,25.5,8,1.21,35.09,SA1
7,8,Al-Fayha FC,26.0,28.1,10,1.16,30.28,SA1
8,9,Abha Club,36.0,27.8,10,764,27.50,SA1
9,10,Al-Fateh SC,29.0,27.6,9,876,25.40,SA1


In [143]:
df_champions

,Season,Clubs,League
0,22/23,Al-Ittihad Club,Yelo League
1,21/22,Al-Hilal SFC,Yelo League
2,20/21,Al-Hilal SFC,Yelo League
3,19/20,Al-Hilal SFC,Yelo League
4,18/19,Al-Nassr FC,Yelo League
5,17/18,Al-Hilal SFC,Yelo League
6,16/17,Al-Hilal SFC,Yelo League
7,15/16,Al-Ahli SFC,Yelo League
8,14/15,Al-Nassr FC,Yelo League
9,13/14,Al-Nassr FC,Yelo League


In [144]:
df_attendances.to_csv(f"./TRUSTED/ArabicSoccer/Attendances/Attendances_SPL_2007_2023.csv", encoding='utf-8', index=False)
df_teams.to_csv(f"./TRUSTED/ArabicSoccer/Teams/Teams_Arabia.csv", encoding='utf-8', index=False)
df_champions.to_csv(f"./TRUSTED/ArabicSoccer/Champions/Champions_Arabia_2007_2023.csv", encoding='utf-8', index=False)